# Đồ Án Môn Lập Trình Python cho máy học

# Nhóm 8

*   25410056 - Lã Xuân Hồng - 25410056@ms.uit.edu.vn
*   25410034 - Lê Quang Hoài Đức - 25410034@ms.uit.edu.vn
*   25410150 - Nguyễn Minh Trọng - 25410150@ms.uit.edu.vn
*   25410088 - Trần Thanh Long - 25410088@ms.uit.edu.vn
*   25410104 - Nguyễn Minh Nhật - 25410104@ms.uit.edu.vn

In [176]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
kritanjalijain_amazon_reviews_path = kagglehub.dataset_download('kritanjalijain/amazon-reviews')

print('Data source import complete.')


Data source import complete.


# 1. Chuẩn bị data

In [177]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tarfile # this is to extract the data from that .tgz file

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-reviews/amazon_review_polarity_csv.tgz
/kaggle/input/amazon-reviews/train.csv
/kaggle/input/amazon-reviews/test.csv


In [178]:
# get all of the data out of that .tgz
# amazon_reviews = tarfile.open('/kaggle/input/amazon-reviews/amazon_review_polarity_csv.tgz')
# amazon_reviews.extractall('data')
# amazon_reviews.close()

In [179]:
import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./.config/.last_opt_in_prompt.yaml
./.config/default_configs.db
./.config/.last_update_check.json
./.config/config_sentinel
./.config/.last_survey_prompt.yaml
./.config/active_config
./.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
./.config/gce
./.config/logs/2025.08.11/13.35.45.017566.log
./.config/logs/2025.08.11/13.35.30.203988.log
./.config/logs/2025.08.11/13.35.54.455059.log
./.config/logs/2025.08.11/13.35.02.204179.log
./.config/logs/2025.08.11/13.35.55.205921.log
./.config/logs/2025.08.11/13.35.39.125256.log
./.config/configurations/config_default
./data/amazon_review_polarity_csv/test.csv
./data/amazon_review_polarity_csv/readme.txt
./data/amazon_review_polarity_csv/train.csv


# 2. Khám phá và làm sạch tập data

*   Loại bỏ các giá trị rỗng (null) và không nhất quán
*   Kiểm tra các bản ghi trùng lặp trong dữ liệu
*   Tiền xử lý văn bản, loại bỏ stopwords, tách từ, và áp dụng kỹ thuật stemming (đưa từ về dạng gốc/dạng từ điển)





In [180]:
import pandas as pd # Nhập thư viện pandas để xử lý và phân tích dữ liệu
import re # Nhập module re (biểu thức chính quy) để tìm kiếm và xử lý chuỗi
import string # Nhập module string để làm việc với các hằng số và hàm của chuỗi
import collections # Nhập module collections để sử dụng các kiểu dữ liệu container đặc biệt

import matplotlib.pyplot as plt # Nhập module pyplot từ matplotlib để vẽ đồ thị

import nltk # Nhập thư viện Natural Language Toolkit (NLTK)
from nltk.corpus import stopwords # Nhập corpus stopwords từ NLTK để truy cập danh sách các từ dừng
from nltk.tokenize import word_tokenize # Nhập hàm word_tokenize từ NLTK để tách văn bản thành từ
from nltk.stem import WordNetLemmatizer, SnowballStemmer # Nhập WordNetLemmatizer và SnowballStemmer để đưa từ về dạng gốc

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
train_df = pd.read_csv('/kaggle/input/amazon-reviews/train.csv')
test_df = pd.read_csv('/kaggle/input/amazon-reviews/test.csv')

train_df.columns = ['label', 'title', 'text']
train_df.head()

test_df.columns = ['label', 'title', 'text']
test_df.head()

print(train_df.head())

   label                                              title  \
0      2              The best soundtrack ever to anything.   
1      2                                           Amazing!   
2      2                               Excellent Soundtrack   
3      2  Remember, Pull Your Jaw Off The Floor After He...   
4      2                            an absolute masterpiece   

                                                text  
0  I'm reading a lot of reviews saying that this ...  
1  This soundtrack is my favorite music of all ti...  
2  I truly like this soundtrack and I enjoy video...  
3  If you've played the game, you know how divine...  
4  I am quite sure any of you actually taking the...  


In [182]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3599999 entries, 0 to 3599998
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   label   int64 
 1   title   object
 2   text    object
dtypes: int64(1), object(2)
memory usage: 82.4+ MB


In [183]:
train_df = train_df.head(100000)
test_df = test_df.head(10000)

**Loại bỏ các giá trị rỗng**

In [184]:
def clean_data(df):
    """
    Kiểm tra và xử lý giá trị null, đồng thời kiểm tra loại dữ liệu trong DataFrame.

    Args:
        df (pd.DataFrame): DataFrame cần làm sạch.

    Returns:
        pd.DataFrame: DataFrame đã được làm sạch.
    """
    print("Số lượng giá trị null trước khi xử lý:")
    print(df.isnull().sum())

    # Điền giá trị NaN trong cột 'text' bằng chuỗi rỗng
    if 'text' in df.columns:
        df['text'] = df['text'].fillna('')

    if 'title' in df.columns:
        df['title'] = df['title'].fillna('')

    print("\nSố lượng giá trị null sau khi xử lý:")
    print(df.isnull().sum())

    # Kiểm tra loại dữ liệu
    print("\nLoại dữ liệu của các cột:")
    print(df.dtypes)

    return df

In [185]:
# Áp dụng hàm làm sạch dữ liệu cho train_df và test_df
print("--- Xử lý train_df ---")
train_df = clean_data(train_df)

print("\n--- Xử lý test_df ---")
test_df = clean_data(test_df)

--- Xử lý train_df ---
Số lượng giá trị null trước khi xử lý:
label    0
title    5
text     0
dtype: int64

Số lượng giá trị null sau khi xử lý:
label    0
title    0
text     0
dtype: int64

Loại dữ liệu của các cột:
label     int64
title    object
text     object
dtype: object

--- Xử lý test_df ---
Số lượng giá trị null trước khi xử lý:
label    0
title    2
text     0
dtype: int64

Số lượng giá trị null sau khi xử lý:
label    0
title    0
text     0
dtype: int64

Loại dữ liệu của các cột:
label     int64
title    object
text     object
dtype: object


**Loại bỏ dữ liệu trùng lặp**

In [186]:
def remove_duplicates(df):
    """
    Kiểm tra và loại bỏ các bản ghi trùng lặp trong DataFrame.

    Args:
        df (pd.DataFrame): DataFrame cần xử lý.

    Returns:
        pd.DataFrame: DataFrame sau khi đã loại bỏ các bản ghi trùng lặp.
    """
    print(f"Số lượng bản ghi trước khi loại bỏ trùng lặp: {len(df)}")

    # Kiểm tra và loại bỏ các bản ghi trùng lặp
    df_cleaned = df.drop_duplicates()

    print(f"Số lượng bản ghi sau khi loại bỏ trùng lặp: {len(df_cleaned)}")

    return df_cleaned

In [187]:
# Áp dụng hàm remove_duplicates cho train_df và test_df
print("--- Xử lý train_df ---")
train_df = remove_duplicates(train_df)

print("\n--- Xử lý test_df ---")
test_df = remove_duplicates(test_df)

--- Xử lý train_df ---
Số lượng bản ghi trước khi loại bỏ trùng lặp: 100000
Số lượng bản ghi sau khi loại bỏ trùng lặp: 100000

--- Xử lý test_df ---
Số lượng bản ghi trước khi loại bỏ trùng lặp: 10000
Số lượng bản ghi sau khi loại bỏ trùng lặp: 10000


**Xóa ký tự không mong muốn**

In [188]:
def clean_text(text):
    """
    Làm sạch văn bản bằng cách:
    - Loại bỏ các ký tự đặc biệt và số.
    - Chuyển đổi sang chữ thường.
    - Loại bỏ dấu câu.
    - Loại bỏ khoảng trắng thừa.
    - Thay thế một hoặc nhiều khoảng trắng bằng một khoảng trắng duy nhất.
    - Loại bỏ khoảng trắng ở đầu/cuối.

    Args:
        text (str): Chuỗi văn bản cần làm sạch.

    Returns:
        str: Chuỗi văn bản đã được làm sạch.
    """
    # Loại bỏ các ký tự đặc biệt và số
    # Sử dụng regex để giữ lại chỉ các chữ cái (tiếng Anh và tiếng Việt có dấu) và dấu cách
    text = re.sub(r'[^A-Za-zÀ-ú ]+', '', text)
    # Chuyển đổi sang chữ thường
    text = text.lower()
    # loại bỏ dấu câu
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Loại bỏ khoảng trắng thừa
    # Thay thế một hoặc nhiều khoảng trắng bằng một khoảng trắng duy nhất và loại bỏ khoảng trắng ở đầu/cuối
    text = re.sub(r'\s+', ' ', text).strip()
    return text # Trả về chuỗi văn bản đã được làm sạch

In [189]:
# Áp dụng hàm clean_text cho train_df và test_df
print("--- Làm sạch train_df ---")
train_df['cleaned_title'] = train_df['title'].apply(clean_text)
train_df['cleaned_text'] = train_df['text'].apply(clean_text)
train_df.head()

print("\n--- Làm sạch test_df ---")
test_df['cleaned_title'] = test_df['title'].apply(clean_text)
test_df['cleaned_text'] = test_df['text'].apply(clean_text)
test_df.head()

--- Làm sạch train_df ---

--- Làm sạch test_df ---


,label,title,text,cleaned_title,cleaned_text
0,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...,one of the best game music soundtracks for a g...,despite the fact that i have only played a sma...
1,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...,batteries died within a year,i bought this charger in jul and it worked ok ...
2,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...,works fine but maha energy is better,check out maha energys website their powerex m...
3,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...,great for the nonaudiophile,reviewed quite a bit of the combo players and ...
4,1,DVD Player crapped out after one year,I also began having the incorrect disc problem...,dvd player crapped out after one year,i also began having the incorrect disc problem...


**Tokenize text thành từng từ riêng lẻ**

In [190]:
def tokenize_text(text):
    """
    Tách văn bản thành các token (từ).

    Args:
        text (str): Chuỗi văn bản cần tách token.

    Returns:
        list: Danh sách các token.
    """
    # Sử dụng word_tokenize của NLTK để tách văn bản thành các token
    if isinstance(text, str):
      return word_tokenize(text)
    else:
      return text

In [191]:
print("--- Tokenize train_df ---")
train_df['tokenized_title'] = train_df['cleaned_title'].apply(tokenize_text)
train_df['tokenized_text'] = train_df['cleaned_text'].apply(tokenize_text)
train_df.head()

print("\n--- Làm sạch test_df ---")
test_df['tokenized_title'] = test_df['cleaned_title'].apply(tokenize_text)
test_df['tokenized_text'] = test_df['cleaned_text'].apply(tokenize_text)
test_df.head()

--- Tokenize train_df ---

--- Làm sạch test_df ---


,label,title,text,cleaned_title,cleaned_text,tokenized_title,tokenized_text
0,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...,one of the best game music soundtracks for a g...,despite the fact that i have only played a sma...,"[one, of, the, best, game, music, soundtracks,...","[despite, the, fact, that, i, have, only, play..."
1,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...,batteries died within a year,i bought this charger in jul and it worked ok ...,"[batteries, died, within, a, year]","[i, bought, this, charger, in, jul, and, it, w..."
2,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...,works fine but maha energy is better,check out maha energys website their powerex m...,"[works, fine, but, maha, energy, is, better]","[check, out, maha, energys, website, their, po..."
3,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...,great for the nonaudiophile,reviewed quite a bit of the combo players and ...,"[great, for, the, nonaudiophile]","[reviewed, quite, a, bit, of, the, combo, play..."
4,1,DVD Player crapped out after one year,I also began having the incorrect disc problem...,dvd player crapped out after one year,i also began having the incorrect disc problem...,"[dvd, player, crapped, out, after, one, year]","[i, also, began, having, the, incorrect, disc,..."


**Loại bỏ stopwords**

In [192]:
def remove_stopwords(tokens):
    """
    Loại bỏ các từ dừng (stopwords) tiếng Anh khỏi danh sách tokens.

    Args:
        filtered_tokens (list): Danh sách các token cần xử lý.

    Returns:
        list: Danh sách các token sau khi đã loại bỏ stopwords.
    """
    if not isinstance(tokens, list):
      return tokens
    else:
      # Tải danh sách các từ dừng tiếng Anh từ NLTK
      stop_words = set(stopwords.words('english'))
      # Lọc bỏ các từ dừng khỏi danh sách tokens
      filtered_tokens = [word for word in tokens if word not in stop_words]
      # Trả về danh sách các token đã lọc
      return filtered_tokens

In [193]:
# Áp dụng hàm remove_stopwords cho train_df và test_df
print("--- Xóa stopwords train_df ---")
train_df['no_stopwords_title'] = train_df['tokenized_title'].apply(remove_stopwords)
train_df['no_stopwords_text'] = train_df['tokenized_text'].apply(remove_stopwords)
train_df.head()

print("\n--- Xóa stopwords test_df ---")
test_df['no_stopwords_title'] = test_df['tokenized_title'].apply(remove_stopwords)
test_df['no_stopwords_text'] = test_df['tokenized_text'].apply(remove_stopwords)
test_df.head()

--- Xóa stopwords train_df ---

--- Xóa stopwords test_df ---


,label,title,text,cleaned_title,cleaned_text,tokenized_title,tokenized_text,no_stopwords_title,no_stopwords_text
0,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...,one of the best game music soundtracks for a g...,despite the fact that i have only played a sma...,"[one, of, the, best, game, music, soundtracks,...","[despite, the, fact, that, i, have, only, play...","[one, best, game, music, soundtracks, game, di...","[despite, fact, played, small, portion, game, ..."
1,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...,batteries died within a year,i bought this charger in jul and it worked ok ...,"[batteries, died, within, a, year]","[i, bought, this, charger, in, jul, and, it, w...","[batteries, died, within, year]","[bought, charger, jul, worked, ok, design, nic..."
2,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...,works fine but maha energy is better,check out maha energys website their powerex m...,"[works, fine, but, maha, energy, is, better]","[check, out, maha, energys, website, their, po...","[works, fine, maha, energy, better]","[check, maha, energys, website, powerex, mhcf,..."
3,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...,great for the nonaudiophile,reviewed quite a bit of the combo players and ...,"[great, for, the, nonaudiophile]","[reviewed, quite, a, bit, of, the, combo, play...","[great, nonaudiophile]","[reviewed, quite, bit, combo, players, hesitan..."
4,1,DVD Player crapped out after one year,I also began having the incorrect disc problem...,dvd player crapped out after one year,i also began having the incorrect disc problem...,"[dvd, player, crapped, out, after, one, year]","[i, also, began, having, the, incorrect, disc,...","[dvd, player, crapped, one, year]","[also, began, incorrect, disc, problems, ive, ..."


**Chuẩn hóa từ**

In [194]:
def normalize_token(tokens):
    """
    Chuẩn hóa danh sách tokens bằng cách áp dụng Snowball Stemmer tiếng Anh cho từng token.

    Args:
        tokens (list): Danh sách các token cần chuẩn hóa.

    Returns:
        list: Danh sách các token sau khi đã chuẩn hóa.
    """
    if not isinstance(tokens, list):
      return tokens
    else:
      # Khởi tạo Snowball Stemmer cho tiếng Anh
      stemmer = SnowballStemmer("english")
      # Áp dụng stemmer cho từng token trong danh sách và trả về danh sách mới
      normalized_tokens = [stemmer.stem(word) for word in tokens]
      return normalized_tokens

In [195]:
# Áp dụng hàm normalize_text cho train_df và test_df
print("--- Chuẩn hóa từ cho train_df ---")
train_df['normalized_title'] = train_df['no_stopwords_title'].apply(normalize_token)
train_df['normalized_text'] = train_df['no_stopwords_text'].apply(normalize_token)

print("\n--- Chuẩn hóa từ cho test_df ---")
test_df['normalized_title'] = test_df['no_stopwords_title'].apply(normalize_token)
test_df['normalized_text'] = test_df['no_stopwords_text'].apply(normalize_token)

--- Chuẩn hóa từ cho train_df ---

--- Chuẩn hóa từ cho test_df ---


In [196]:
train_df.head()

,label,title,text,cleaned_title,cleaned_text,tokenized_title,tokenized_text,no_stopwords_title,no_stopwords_text,normalized_title,normalized_text
0,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...,the best soundtrack ever to anything,im reading a lot of reviews saying that this i...,"[the, best, soundtrack, ever, to, anything]","[im, reading, a, lot, of, reviews, saying, tha...","[best, soundtrack, ever, anything]","[im, reading, lot, reviews, saying, best, game...","[best, soundtrack, ever, anyth]","[im, read, lot, review, say, best, game, sound..."
1,2,Amazing!,This soundtrack is my favorite music of all ti...,amazing,this soundtrack is my favorite music of all ti...,[amazing],"[this, soundtrack, is, my, favorite, music, of...",[amazing],"[soundtrack, favorite, music, time, hands, int...",[amaz],"[soundtrack, favorit, music, time, hand, inten..."
2,2,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...,excellent soundtrack,i truly like this soundtrack and i enjoy video...,"[excellent, soundtrack]","[i, truly, like, this, soundtrack, and, i, enj...","[excellent, soundtrack]","[truly, like, soundtrack, enjoy, video, game, ...","[excel, soundtrack]","[truli, like, soundtrack, enjoy, video, game, ..."
3,2,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine...",remember pull your jaw off the floor after hea...,if youve played the game you know how divine t...,"[remember, pull, your, jaw, off, the, floor, a...","[if, youve, played, the, game, you, know, how,...","[remember, pull, jaw, floor, hearing]","[youve, played, game, know, divine, music, eve...","[rememb, pull, jaw, floor, hear]","[youv, play, game, know, divin, music, everi, ..."
4,2,an absolute masterpiece,I am quite sure any of you actually taking the...,an absolute masterpiece,i am quite sure any of you actually taking the...,"[an, absolute, masterpiece]","[i, am, quite, sure, any, of, you, actually, t...","[absolute, masterpiece]","[quite, sure, actually, taking, time, read, pl...","[absolut, masterpiec]","[quit, sure, actual, take, time, read, play, g..."


In [197]:
test_df.head()

,label,title,text,cleaned_title,cleaned_text,tokenized_title,tokenized_text,no_stopwords_title,no_stopwords_text,normalized_title,normalized_text
0,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...,one of the best game music soundtracks for a g...,despite the fact that i have only played a sma...,"[one, of, the, best, game, music, soundtracks,...","[despite, the, fact, that, i, have, only, play...","[one, best, game, music, soundtracks, game, di...","[despite, fact, played, small, portion, game, ...","[one, best, game, music, soundtrack, game, did...","[despit, fact, play, small, portion, game, mus..."
1,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...,batteries died within a year,i bought this charger in jul and it worked ok ...,"[batteries, died, within, a, year]","[i, bought, this, charger, in, jul, and, it, w...","[batteries, died, within, year]","[bought, charger, jul, worked, ok, design, nic...","[batteri, die, within, year]","[bought, charger, jul, work, ok, design, nice,..."
2,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...,works fine but maha energy is better,check out maha energys website their powerex m...,"[works, fine, but, maha, energy, is, better]","[check, out, maha, energys, website, their, po...","[works, fine, maha, energy, better]","[check, maha, energys, website, powerex, mhcf,...","[work, fine, maha, energi, better]","[check, maha, energi, websit, powerex, mhcf, c..."
3,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...,great for the nonaudiophile,reviewed quite a bit of the combo players and ...,"[great, for, the, nonaudiophile]","[reviewed, quite, a, bit, of, the, combo, play...","[great, nonaudiophile]","[reviewed, quite, bit, combo, players, hesitan...","[great, nonaudiophil]","[review, quit, bit, combo, player, hesit, due,..."
4,1,DVD Player crapped out after one year,I also began having the incorrect disc problem...,dvd player crapped out after one year,i also began having the incorrect disc problem...,"[dvd, player, crapped, out, after, one, year]","[i, also, began, having, the, incorrect, disc,...","[dvd, player, crapped, one, year]","[also, began, incorrect, disc, problems, ive, ...","[dvd, player, crap, one, year]","[also, began, incorrect, disc, problem, ive, r..."
